In [1]:
# todo:
# - understand how the training data text is represented for chain of thought ot 3SUM sequences
# - function for evaluating accuracy
# - understand decoder implementation
# - function to print rank-n for layer m and then iterate over using this


# experiments:
# - write the claims of the paper: COT is uninterpretable as it can be hidden, but this hidden COT still increases performs
# - see where it is being replaced with '.' and where it is just the numbers
# - check this across layer, across rank
# - is this gradual or not gradual?
# - check if these decoded sequences from different ranks and layers are in anyway simialr to the actual trianing dataset.
# - write the punchline: you only see the decoded text of COT, but not what is there in the rest of the logits as information.

In [2]:
from data_utils import *
from nethook import *
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from utils import InputEmbedCausalTransformer
import json
import numpy as np
import os 
import pandas as pd
import torch

/home/loki/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
MODEL_PATH = "/home/loki/projects/filler_tokens/checkpoints/2024-07-21-17-minidata-checkpoint-final/model_weights.pt"
CONFIG_FILE = "/home/loki/projects/filler_tokens/configs/llama_d384l4h6.json"

In [4]:
train_df = pd.read_csv('data/match3data_trainset_2024-08-10.csv', header=None, names=["text"])
test_df = pd.read_csv('data/match3data_testset_2024-08-10.csv', header=None, names=["text"])

In [5]:
train_df.iloc[1].text

' 072 673 359 052 321 177 612 692 768 730 P . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . A True'

In [6]:
test_df.iloc[1].text

' 441 789 625 581 232 698 085 850 605 779 P 0- 1 0- 6 0- 9 0- 3 0- 9 6- 2 0- 2 8- 4 0- 1 2- 4 3- 6 4- 1 5- 3 1- 6 7- 9 8- 4 9- 4 3- 6 2- 7 5- 2 6- 0 2- 5 8- 2 9- 9 3- 7 5- 1 6- 5 7- 9- 3- 8 9- 0 4- 8 4- 7 4- 0 4- 8 9- 9 6- 3 7- 4 5- 9 5- 7 6- 8 6- 0 6- 5 8- 5 7- 5 8- 7 9- 7 A True'

In [7]:
test_set = Match3VectorDataset(test_df, 3, 10, 10, 'P')
print(test_set.input_dim)

validate encodings
raw input 0  655 748 451 416 889 352 187 278 880 014 P . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . A False
encoded sample 0 {'input_ids': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float16), 'labels': tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100,    3,    3,
           3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
           3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
           3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
           3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
 

In [44]:
class BlockOutputWrapper(torch.nn.Module):
    def __init__(self, block, unembed_matrix, norm):
        super().__init__()
        self.block = block
        self.unembed_matrix = unembed_matrix
        self.norm = norm
        self.block_output_unembedded = None

    def forward(self, *args, **kwargs):
        output = self.block(*args, **kwargs)
        if isinstance(output, tuple):
            self.block_output_unembedded = self.unembed_matrix(self.norm(output[0]))
            return output
        else:
            self.block_output_unembedded = self.unembed_matrix(self.norm(output))
            return output

    def reset_block_output(self):
        self.block_output_unembedded = None

class LlamaHelper:
    def __init__(self, config_file, model_path, test_set):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        config = AutoConfig.from_pretrained(config_file)
        model = InputEmbedCausalTransformer(AutoModelForCausalLM.from_config(config), test_set.input_dim)
        state_dict = torch.load(model_path)
        model.load_state_dict(state_dict, strict=False)
        model = model.to(self.device)
        self.model = model
        self.word_index_map = test_set.word_index_map
        self.data_len = test_set.data_len
        self.mod = test_set.mod
        
        for i, layer in enumerate(self.model.base_model.model.layers):
            self.model.base_model.model.layers[i] = BlockOutputWrapper(layer, self.model.base_model.lm_head, self.model.base_model.model.norm)

    def decode_tensors(self, sequence):
        decoded_sequence = []
        marker_found = False

        for token in sequence:
            token = token.item()
            if token == -100:
                if not marker_found:
                    decoded_sequence.append("[MASK]")
                continue
            elif token == 0:
                decoded_sequence.append("[EOS]")
                break  # Stop decoding after EOS
            elif token < len(self.word_index_map):
                # Regular word
                word = list(self.word_index_map.keys())[list(self.word_index_map.values()).index(token)]
                decoded_sequence.append(word)
                if word in ["A", "P"]:
                    marker_found = True
            else:
                # Handle digit labels
                offset = len(self.word_index_map)
                if token < offset + self.data_len * 2:
                    # Tuple index encoding
                    idx = (token - offset) % self.data_len
                    tuple_pos = (token - offset) // self.data_len
                    decoded_sequence.append(f"{tuple_pos}-{idx}")
                else:
                    # Single digit or digit in tuple
                    char_pos = (token - offset - self.data_len * 2) // self.mod
                    digit = (token - offset - self.data_len * 2) % self.mod
                    if char_pos == 0 or len(decoded_sequence) == 0 or not decoded_sequence[-1][-1].isdigit():
                        decoded_sequence.append(str(digit))
                    else:
                        decoded_sequence[-1] += str(digit)

        return " ".join(decoded_sequence)


    def get_logits(self, inputs):
        with torch.no_grad():
            input_ids = inputs['input_ids'].float().to(self.device).unsqueeze(0)
            out = self.model(input_ids)
            return out.logits

    def set_add_attn_output(self, layer, add_output):
        self.model.base_model.model.layers[layer].attn_add_tensor(add_output)

    def get_attn_activations(self, layer):
        return self.model.base_model.model.layers[layer].get_attn_activations()

    def reset_all_layers(self):
        for layer in self.model.base_model.model.layers:
            layer.reset_block_output()

    def print_logit_progression(self, inputs, layer_number=None, rank=1, sequence_length=None):
        self.reset_all_layers()
        logits = self.get_logits(inputs)

        num_layers = len(self.model.base_model.model.layers)
        seq_length = inputs['input_ids'].size(1) if sequence_length is None else min(sequence_length, inputs['input_ids'].size(1))

        def process_layer(layer_idx):
            tokens = []
            values = []
            layer = self.model.base_model.model.layers[layer_idx] if layer_idx < num_layers else None
            for j in range(seq_length):
                if layer and layer.block_output_unembedded is not None:
                    layer_logits = layer.block_output_unembedded[0][j]
                elif layer_idx == num_layers:  # Final output
                    layer_logits = logits[0][j]
                else:
                    continue

                val, idx = torch.topk(layer_logits, rank)
                tokens.append(self.decode_tensors(idx[-1].unsqueeze(-1)).strip())
                values.append(val[-1].item())

            return tokens, values

        if layer_number is not None:
            if layer_number > num_layers:
                print(f"Error: Layer number {layer_number} is out of range. Max layer is {num_layers}.")
                return

            tokens, values = process_layer(layer_number)
            layer_name = f"h{layer_number}_out" if layer_number < num_layers else "h_out"
            print(f"{rank}th highest logit progression for {layer_name}:")
            print("".join(tokens))
            print(" ".join([f"{v:.2f}" for v in values]))
        else:
            print(f"{rank}th highest logit progression:")
            for i in range(num_layers + 1):
                tokens, values = process_layer(i)
                layer_name = f"h{i}_out" if i < num_layers else "h_out"
                print(f"{layer_name:<5}: " + "".join(tokens))


In [45]:
model = LlamaHelper(CONFIG_FILE, MODEL_PATH, test_set)

In [51]:
test_df.iloc[0].text

' 655 748 451 416 889 352 187 278 880 014 P . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . A False'

In [53]:
for i in range(4):
    model.print_logit_progression(test_set[i], rank=1, sequence_length=len(test_set[i]['input_ids']))
    model.print_logit_progression(test_set[i], rank=2, sequence_length=len(test_set[i]['input_ids']))
    print("\n")

1th highest logit progression:
h0_out: 0-10-80-860-0..0-00-00-0.......................................................
h1_out: .................................................................
h2_out: .................................................................
h3_out: .................................................................
h_out: .................................................................
2th highest logit progression:
h0_out: 0-060-0160-00-0...0-1AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
h1_out: 0-10-00-00-00-00-00-00-00-00-00-1AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
h2_out: 0-00-00-00-00-00-00-00-00-00-00-1AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
h3_out: 0-00-00-00-00-00-00-00-00-00-00-0AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
h_out: 0-00-00-00-00-00-00-00-00-00-00-0AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


1th highest logit progression:
h0_out: .0-80-00-0..0-00-00-00-00-140-240-240-240-240-2